In [1]:
!pip install -r requeminents.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 KB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215267 sha256=9c970b60d15ba1f426c27aa919a3de5b3f884514ea8cb9f2751f0cf2a99e572d
  Stored in directory: /home/dmitriy/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pytelegrambotapi


In [2]:
import telebot
from telebot import types
import logging
import gc
from game import *
from model import *

In [37]:
def game_st(agent):
  results = [agent.game_now() for i in range(10000)]
  game_stats = {k: results.count(k)/100 for k in ["X", "0", '-']}
  print("Статистика обучения: {}".format(game_stats))

def to_numb(x):
  x = x.lower()
  if len(x) != 2 or x[1].isalpha() or int(x[1]) > 3 :
    return 10
  if x[0] == "a":
    return int(x[1]) - 1
  if x[0] == "b":
    return 2 + int(x[1])
  if x[0] == "c":
    return 5 + int(x[1])
  return 10

In [6]:
agent = Agent007(Crosses_zeros)
print('Before:')
game_st(agent)

agent.learn(100000)
print('After:')
game_st(agent)

Before:
Статистика обучения: {'X': 58.39, '0': 28.95, '-': 12.66}
After:
Статистика обучения: {'X': 0.0, '0': 0.0, '-': 100.0}


In [39]:
player = dict()
game = dict()

def generate_buttoms(table):
   rows = ['a', 'b', 'c']
   keyboard = types.InlineKeyboardMarkup()
   for now in range(9):
      if table[now] == 1337:
         value = rows[now // 3] + str(now % 3 + 1)
         button = types.InlineKeyboardButton(text=value, callback_data=value)
         keyboard.add(button)
        
   return keyboard


bot = telebot.TeleBot('1818851136:AAHzS5ZKjx_GNfe5V_LhpdnB95lWlXW9E08')
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
  if message.text.lower() == "привет":
    bot.send_message(message.from_user.id, "Привет, хочешь поиграть?")
    bot.register_next_step_handler_by_chat_id(message.chat.id, continue_bot)
  elif message.text == "/help":
    bot.send_message(message.from_user.id, "Напиши привет")
  else:
    bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")

def continue_bot(message):
  if (message.text).lower() == 'да':
    figure(message)
  else:
    bot.send_message(message.from_user.id, "Тогда пока!")

def figure(message):
      keyboard = types.InlineKeyboardMarkup() #наша клавиатура
      key_yes = types.InlineKeyboardButton(text='X', callback_data="1") #кнопка «Да»
      keyboard.add(key_yes) #добавляем кнопку в клавиатуру
      key_yes2 = types.InlineKeyboardButton(text='0', callback_data="0")
      keyboard.add(key_yes2)
      question = 'Что ты выберешь?'
      bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if len(call.data) == 2:
       call.message.text = call.data
       agent_play(call.message)
       return
    if len(call.data) != 1:
       bot.reply_to(call.message, "Некорректный ход")
    bot.send_message(call.message.chat.id, 'Хороший выбор! Если хочешь, чтобы я тебя понял, свой ход делай с помощью команд: a1, a2, a3, b1, b2, b3, c1, c2, c3, где a, b, c - строки, а 1, 2, 3 - столбцы ')
    global player, game
    game[call.message.chat.id] = Crosses_zeros()
    player[call.message.chat.id] = int(call.data)
    if  player[call.message.chat.id] == 0:
        game[call.message.chat.id].do_action(agent.select_action_for_interactive_game(game[call.message.chat.id], 1))
        bot.send_message(call.message.chat.id, game[call.message.chat.id].table_to_string(), parse_mode="HTML")
    
    keyboard = generate_buttoms(game[call.message.chat.id].table)
    question = "Выбери свой путь"
    bot.send_message(call.message.chat.id, text=question, reply_markup=keyboard)
    bot.register_next_step_handler_by_chat_id(call.message.chat.id, agent_play)

def agent_play(message):
    global player, game
    now = to_numb(message.text)
    table = game[message.chat.id].table
    if now < 0 or now > 8 or table[now] != 1337:
      bot.reply_to(message, "Некорректный ход")
    else:
      table[now] = player[message.chat.id]
      game[message.chat.id].do_action(table)
      if game[message.chat.id].playable():
        game[message.chat.id].do_action(agent.select_action_for_interactive_game(game[message.chat.id], 1 - player[message.chat.id]))
      bot.send_message(message.chat.id, game[message.chat.id].table_to_string(), parse_mode="HTML")
    if game[message.chat.id].playable():
      keyboard = generate_buttoms(game[message.chat.id].table)
      question = "Выбери свой путь"
      bot.send_message(message.chat.id, text=question, reply_markup=keyboard)
      bot.register_next_step_handler_by_chat_id(message.chat.id, agent_play)
    else:
      if game[message.chat.id].check_winner() == 'X':
        bot.send_message(message.chat.id, 'Выиграл X!!!', parse_mode="HTML")
      elif game[message.chat.id].check_winner() == '0':
        bot.send_message(message.chat.id, 'Выиграл 0!!!', parse_mode="HTML")
      else:
        bot.send_message(message.chat.id, 'Ничья!!!', parse_mode="HTML")
      bot.clear_step_handler_by_chat_id(message.chat.id)


bot.polling(none_stop=True, interval=0)
